In [2]:
import pandas as pd
import numpy as np

In [3]:
% ls ../

data.csv             task2/               train.csv
data_preprocess.csv  task3/
task1/               test.csv


In [7]:
data = pd.read_csv('../data_preprocess.csv')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4752 entries, 0 to 4751
Data columns (total 79 columns):
regional_mobility                             4752 non-null float64
is_high_user                                  4752 non-null float64
avg_consume_less_12_valid_month               4752 non-null float64
reg_preference_for_trad                       4752 non-null float64
status                                        4752 non-null float64
low_volume_percent                            4752 non-null float64
middle_volume_percent                         4752 non-null float64
take_amount_in_later_12_month_highest         4752 non-null float64
trans_amount_increase_rate_lately             4752 non-null float64
trans_activity_month                          4752 non-null float64
trans_activity_day                            4752 non-null float64
transd_mcc                                    4752 non-null float64
trans_days_interval_filter                    4752 non-null float64
trans_day

In [9]:
# IV值的计算
import math
from scipy import stats
from sklearn.utils.multiclass import type_of_target
import numpy as np
def woe(X, y, event=1):  
    res_woe = []
    iv_dict = {}
    for feature in X.columns:
        x = X[feature].values
        # 1) 连续特征离散化
        if type_of_target(x) == 'continuous':
            x = discrete(x)
        # 2) 计算该特征的woe和iv
        # woe_dict, iv = woe_single_x(x, y, feature, event)
        woe_dict, iv = woe_single_x(x, y, feature, event)
        iv_dict[feature] = iv
        res_woe.append(woe_dict) 
        
    return iv_dict
def discrete(x):
    # 使用5等分离散化特征
    res = np.zeros(x.shape)
    for i in range(5):
        #stats.scoreatpercentile([1,2,3,4,5],50)    # 计算输入序列中给定百分位数的分数 返回值是3
        point1 = stats.scoreatpercentile(x, i * 20)
        point2 = stats.scoreatpercentile(x, (i + 1) * 20)
        x1 = x[np.where((x >= point1) & (x <= point2))]  #将x分到[0,20],[20,40],[40,60],[60,80],[80,100]5个区间进行离散化
        mask = np.in1d(x, x1) #测试一维数组的每个元素是否也存在于第二个数组中
        res[mask] = i + 1    # 将[i, i+1]块内的值标记成i+1
    return res

def woe_single_x(x, y, feature,event = 1):
    # event代表预测正例的标签
    event_total = (y == event).sum()  #预测为正例的数量
    non_event_total = y.shape[-1] - event_total #预测为负例的数量
    
    iv = 0
    woe_dict = {}
    for x1 in set(x):    # 遍历各个特征标签
        y1 = y.reindex(np.where(x == x1)[0])
        event_count = sum(y1 == event)  
        non_event_count = y1.shape[-1] - event_count
        
        rate_event = event_count / event_total  
        rate_non_event = non_event_count / non_event_total
        
        if rate_event == 0:
            rate_event = 0.0001
            # woei = -20
        elif rate_non_event == 0:
            rate_non_event = 0.0001
            # woei = 20
        woei = math.log(rate_event / rate_non_event) #表示第i组中正例/负例比值
        woe_dict[x1] = woei
        iv += (rate_event - rate_non_event) * woei
    return woe_dict, iv

In [13]:
y = data['status']
x = data.drop(['status'],axis=1)

In [16]:
y.shape

(4752,)

In [17]:
iv_dict = woe(x,y)
iv = pd.DataFrame(sorted(iv_dict.items(), key = lambda x:x[1],reverse = True),columns=['Var','Iv'])
iv

/Users/liuxiaoying/anaconda3/envs/my_env/lib/python3.6/site-packages/scipy/stats/stats.py:1633: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


,Var,Iv
0,historical_trans_amount,2.497334
1,trans_amount_3_month,2.324892
2,pawns_auctions_trusts_consume_last_6_month,1.973832
3,repayment_capability,1.947715
4,abs,1.438627
5,consfin_avg_limit,1.384010
6,consume_mini_time_last_1_month,1.120906
7,loans_avg_limit,0.909205
8,max_cumulative_consume_later_1_month,0.899189
9,loans_score,0.868281
